In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#working with simple data ,(no preprocessing done now),suppose these are the 4 documemnts
textFiles = ["Doc1.txt","Doc2.txt","Doc3.txt"]

In [3]:
import nltk
import string
from nltk.stem.porter import PorterStemmer
import codecs

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

token_dict = {}
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        item = wordnet_lemmatizer.lemmatize(item)
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for file_name in textFiles:
    with codecs.open(file_name, "r",encoding='utf-8', errors='ignore') as f:  
        text = f.read().encode('utf-8').strip()
        text = text.replace('\r', ' ')
        text = text.replace('\n', ' ')
        lowers = text.lower().strip()
        no_punctuation = lowers.translate(None, string.punctuation)
        token_dict[file_name] = no_punctuation

In [25]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',min_df=3)
tfs = tfidf.fit_transform(token_dict.values())
feature_names = tfidf.get_feature_names()
for col in tfs.nonzero()[1]:
    print feature_names[col], ' - ', tfs[1, col]
len(feature_names)

produc  -  0.0640184399664
carbon  -  0.192055319899
nanotub  -  0.256073759866
chemic  -  0.128036879933
vapor  -  0.128036879933
deposit  -  0.128036879933
specif  -  0.0640184399664
temperatur  -  0.640184399664
use  -  0.320092199832
howev  -  0.128036879933
control  -  0.0640184399664
addit  -  0.128036879933
high  -  0.0640184399664
nano  -  0.0640184399664
need  -  0.0640184399664
improv  -  0.0640184399664
align  -  0.0640184399664
applic  -  0.128036879933
devic  -  0.128036879933
field  -  0.0640184399664
0007  -  0.0640184399664
method  -  0.192055319899
ha  -  0.0640184399664
growth  -  0.384110639799
0008  -  0.0640184399664
veri  -  0.0640184399664
therefor  -  0.0640184399664
heretofor  -  0.0640184399664
unaddress  -  0.0640184399664
exist  -  0.0640184399664
art  -  0.0640184399664
address  -  0.0640184399664
aforement  -  0.0640184399664
defici  -  0.0640184399664
produc  -  0.0640184399664
carbon  -  0.192055319899
nanotub  -  0.256073759866
chemic  -  0.128036879933

34

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [27]:
#finding cosine similarity with doc1 with other documents
#cosine_similarities = cosine_similarity(tfs[0:1], tfs)

cosine_similarities = linear_kernel(tfs[0:1], tfs)



In [28]:
cosine_similarities

array([[ 1.        ,  0.75394762,  0.74233902]])

In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0)
tfidf_matrix =  tf.fit_transform(token_dict.values())
from sklearn.metrics.pairwise import linear_kernel
 
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [188]:
for index, score in find_similar(tfs, 0):
       print score, docs[index]

0.232114557988 <open file 'Doc2.txt', mode 'r' at 0x113db7780>
0.185614545175 <open file 'Doc3.txt', mode 'r' at 0x113db7810>


In [189]:
from gensim import corpora, models, similarities
from collections import defaultdict


/Library/Python/2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [193]:
stoplist = set(['is', 'how'])

texts = [[word.lower() for word in document.split()
          if word.lower() not in stoplist]
         for document in documents]

print texts
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
dictionary = corpora.Dictionary(texts)

# doc2bow counts the number of occurences of each distinct word,
# converts the word to its integer word id and returns the result
# as a sparse vector

corpus = [dictionary.doc2bow(text) for text in texts]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
doc = "giraffe poop car murderer"
vec_bow = dictionary.doc2bow(doc.lower().split())

# convert the query to LSI space
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])

# perform a similarity query against the corpus
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

print sims


[['doc1.txt'], ['doc3.txt'], ['doc2.txt']]


ValueError: failed to create intent(cache|hide)|optional array-- must have defined dimensions but got (0,)